In [ ]:
import json
import requests
import pymysql
import pandas as pd
import sqlalchemy # install if needed
from sqlalchemy import create_engine

def lambda_handler(event, context):
 
    #for cities data0
    schema = "gans"
    host = "**your host**.eu-central-1.rds.amazonaws.com"
    user = "****"
    password ="****"
    port = 3306
    con = f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}"
    
    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}', echo=False)
    cities_df = pd.read_sql_table("cities", con=engine)
    
    cities = cities_df["city"].to_list()
    
    #creating function for weather 
     
    def get_weather_data(cities):
        weather_list = []
        url = f"http://api.openweathermap.org/data/2.5/forecast?q=Berlin&appid={'APIKEY'}&units=metric"
        test = requests.get(url)
        if test.status_code >= 200 and test.status_code <= 299:
            for city in cities:
                url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={'APIKEY'}&units=metric"
                weather = requests.get(url)
                weather_df = pd.json_normalize(weather.json()["list"])
                weather_df["city"] = city
                weather_list.append(weather_df)
                
        else:
            return -1
        weather_combined = pd.concat(weather_list, ignore_index = True)
        return weather_combined

    #rename the tables names

    weather_data = get_weather_data(cities)


    weather_data = weather_data[["pop", "dt_txt", "main.temp", "main.feels_like", "main.humidity", "clouds.all", 
                             "wind.speed", "wind.gust", "city"]]
                             
   
    weather_data.rename(columns = {"pop": "precip_prob", 
                                "dt_txt": "forecast_time", 
                                "main.temp": "temperature",
                                "main.feels_like": "feels_like",
                                "main.humidity": "humidity", 
                                "clouds.all": "cloudiness", 
                                "wind.speed": "wind_speed", 
                                "wind.gust": "wind_gust",}, 
                                 inplace = True)
    weather_data = weather_data.merge(cities_df[["city_id", "city"]], how = "left", on = "city")
    weather_data["forecast_time"] = pd.to_datetime(weather_data["forecast_time"])

    weather_data.to_sql("weathers", con = engine)
   
 #creating function for airport data
    
    def icao_airport_code(latitude, longitude):
        airport_list = []
        # check the length of the latitude and longitude lists to make sure they are equal
        assert len(latitude) == len(longitude)
        # set the API call to get airport data within 50km of the lat and lon being input and show 10 results
        url = "https://aerodatabox.p.rapidapi.com/airports/search/location/51.511142/-0.103869/km/50/10"
        querystring = {"withFlightInfoOnly":"true"}
        headers = {
            "X-RapidAPI-Key": API_key,
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }
        
        test = requests.request("GET", url, headers = headers, params = querystring)
        if test.status_code >= 200 and test.status_code <= 299:
            for i in range(len(latitude)):
            url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitude[i]}/{longitude[i]}/km/50/10"
            querystring = {"withFlightInfoOnly":"true"}
            headers = {
                "X-RapidAPI-Key": API_key, 
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            response = requests.request("GET", url, headers = headers, params = querystring)
            airport_df = pd.json_normalize(response.json()["items"])
            airport_list.append(airport_df)
            
        else:
            return -1
        
        airports_df = pd.concat(airport_list, ignore_index = True)
        return airports_df
    
     
    airport_data.rename(columns = {"name": "airport_name",
                               "municipalityName": "municipality_name",
                               "countryCode": "country_code",
                               "location.lat": "airport_latitude",
                               "location.lon": "airport_longitude"},
                    inplace = True)
    # merge the airport data with the city data to add a city_id column to airport data
    airport_data = airport_data.merge(city_data[["city_id", "city"]], how = "left", left_on = "municipality_name", right_on = "city")
    
    airport_data.reset_index(drop = True, inplace = True)
    airport_data.to_sql("airports", con = engine)
    
    #creating function for flights
    
    def get_flight_data(icao):
        # use the datetime function in python to get today's and tomorrow's date to be used as inputs of the API call
        today = datetime.now().astimezone(timezone("Europe/Berlin")).date()
        tomorrow = (today + timedelta(days = 1))
        arrival_list = []
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/EHAM/{tomorrow}T10:00/{tomorrow}T22:00"
        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true",
                       "withCargo":"false","withPrivate":"false","withLocation":"false"}
        headers = {
            "X-RapidAPI-Key": API_key,
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }
        test = requests.request("GET", url, headers = headers, params = querystring)
        
        if test.status_code >= 200 and test.status_code <= 299:
            for code in icao:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{code}/{tomorrow}T10:00/{tomorrow}T22:00"
            querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false",
                           "withCodeshared":"true","withCargo":"false","withPrivate":"false",
                           "withLocation":"false"}
            headers = {
                "X-RapidAPI-Key": API_key,
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            
            response = requests.request("GET", url, headers = headers, params = querystring)
            arrival_df = pd.json_normalize(response.json()["arrivals"])
            arrival_df["arrival_icao"] = code
            arrival_list.append(arrival_df)
        else:
             return -1
        arrivals_df = pd.concat(arrival_list, ignore_index = True)  
        return arrivals_df   
    
    flight_data.drop(columns = ["codeshareStatus", "isCargo", "movement.scheduledTimeUtc", "movement.quality", 
                             "aircraft.reg", "aircraft.modeS", "callSign", "movement.actualTimeLocal",
                             "movement.actualTimeUtc", "movement.gate", "movement.baggageBelt"], inplace = True)
                             
    flight_data.rename(columns = {"number": "flight_number",
                              "movement.airport.icao": "departure_icao",
                              "movement.airport.iata": "departure_iata",
                              "movement.airport.name": "departure_airport",
                              "movement.scheduledTimeLocal": "scheduled_time",
                              "movement.terminal": "terminal",
                              "aircraft.model": "aircraft_model",
                              "airline.name": "airline"},
                   inplace = True)
    
    flight_data["scheduled_time"] = pd.to_datetime(flight_data["scheduled_time"])
    
    
    flight_df.to_sql("flights",con = engine)
            
    
    
    
    
    
    
    
    
    
    
    
    
    
    